In [16]:
import numpy as np
import pandas as pd
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 26.2034,
	"longitude": 98.2300,
	"start_date": "2018-01-01",
	"end_date": "2023-12-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "wind_speed_10m", "wind_speed_100m", "wind_gusts_10m", "soil_temperature_0_to_7cm", "soil_temperature_7_to_28cm", "soil_temperature_28_to_100cm", "soil_temperature_100_to_255cm"],
	"temperature_unit": "fahrenheit"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(2).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
hourly_rain = hourly.Variables(4).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(5).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(7).ValuesAsNumpy()
hourly_wind_speed_100m = hourly.Variables(8).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(9).ValuesAsNumpy()
hourly_soil_temperature_0_to_7cm = hourly.Variables(10).ValuesAsNumpy()
hourly_soil_temperature_7_to_28cm = hourly.Variables(11).ValuesAsNumpy()
hourly_soil_temperature_28_to_100cm = hourly.Variables(12).ValuesAsNumpy()
hourly_soil_temperature_100_to_255cm = hourly.Variables(13).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s"),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["soil_temperature_0_to_7cm"] = hourly_soil_temperature_0_to_7cm
hourly_data["soil_temperature_7_to_28cm"] = hourly_soil_temperature_7_to_28cm
hourly_data["soil_temperature_28_to_100cm"] = hourly_soil_temperature_28_to_100cm
hourly_data["soil_temperature_100_to_255cm"] = hourly_soil_temperature_100_to_255cm

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)


Coordinates 26.186290740966797°N 98.1907958984375°E
Elevation 1029.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                     date  temperature_2m  relative_humidity_2m  \
0     2018-01-01 00:00:00       46.934601             84.262932   
1     2018-01-01 01:00:00       47.384598             84.001190   
2     2018-01-01 02:00:00       53.864601             71.500198   
3     2018-01-01 03:00:00       61.514603             54.496605   
4     2018-01-01 04:00:00       67.184601             42.902416   
...                   ...             ...                   ...   
52579 2023-12-31 19:00:00       45.584599             81.291443   
52580 2023-12-31 20:00:00       43.694599             85.539665   
52581 2023-12-31 21:00:00       43.064598             86.710709   
52582 2023-12-31 22:00:00       42.884598             87.617844   
52583 2023-12-31 23:00:00       42.794601             87.000603   

       apparent_temperature  precipitation  rain  snowfall  snow_dep

In [17]:
# hourly_dataframe.to_csv(r"D:\OneDrive\Edward Gu\Cornell\CDS\Millennium Project\env_data\Abilene_weather(WEST).csv")
# hourly_dataframe.to_csv(r"D:\OneDrive\Edward Gu\Cornell\CDS\Millennium Project\env_data\DallasFortWorth_weather(NCENTRAL).csv")
# hourly_dataframe.to_csv(r"D:\OneDrive\Edward Gu\Cornell\CDS\Millennium Project\env_data\Austin_weather(SCENTRAL).csv")
hourly_dataframe.to_csv(r"D:\OneDrive\Edward Gu\Cornell\CDS\Millennium Project\env_data\MCAllen_weather(SOUTH).csv")

In [18]:
hourly_dataframe

,date,temperature_2m,relative_humidity_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,wind_speed_10m,wind_speed_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm
0,2018-01-01 00:00:00,46.934601,84.262932,43.649323,0.0,0.0,0.0,0.0,5.351785,7.594208,20.160000,47.654602,53.144600,56.924599,62.864601
1,2018-01-01 01:00:00,47.384598,84.001190,44.566723,0.0,0.0,0.0,0.0,3.877318,7.594208,21.599998,47.834599,52.874599,56.924599,62.774601
2,2018-01-01 02:00:00,53.864601,71.500198,51.732948,0.0,0.0,0.0,0.0,3.096837,6.989936,19.440001,49.454597,52.694599,56.924599,62.774601
3,2018-01-01 03:00:00,61.514603,54.496605,59.681915,0.0,0.0,0.0,0.0,2.036468,3.600000,18.000000,51.974602,52.604599,56.924599,62.774601
4,2018-01-01 04:00:00,67.184601,42.902416,65.147125,0.0,0.0,0.0,0.0,1.835647,2.189795,14.400000,54.854599,52.784599,56.834599,62.774601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2023-12-31 19:00:00,45.584599,81.291443,41.900780,0.0,0.0,0.0,0.0,5.040000,6.162207,16.199999,46.394600,52.244598,56.024601,63.404602
52580,2023-12-31 20:00:00,43.694599,85.539665,39.778122,0.0,0.0,0.0,0.0,5.506941,7.172949,16.919998,45.764599,51.974602,56.024601,63.404602
52581,2023-12-31 21:00:00,43.064598,86.710709,39.186405,0.0,0.0,0.0,0.0,5.154416,7.754637,17.280001,45.224602,51.704597,56.024601,63.404602
52582,2023-12-31 22:00:00,42.884598,87.617844,39.116619,0.0,0.0,0.0,0.0,4.802999,7.754637,17.639999,44.774601,51.434601,56.024601,63.404602
